In [ ]:
import os
import pandas as pd

from rdkit import Chem
import torch

from tqdm import trange
work_dir = r'E:/Coding/jupyter_root/Kaggle/neurips-open-polymer-prediction-2025/polymer_prediction_notebook'
df = pd.read_csv(os.path.join(work_dir, './merge_fill_ffv_mean.csv'))
merge_csv = pd.read_csv(os.path.join(work_dir, 'merge.csv'))

In [ ]:
# Continuous version
def get_atom_edge_index(smiles:pd.Series):
    all_edge_index = []
    batch = []
    atom_offset = 0
    for id, smile in enumerate(smiles):
        mol = Chem.MolFromSmiles(smile)
        num_atoms = mol.GetNumAtoms()
        edges = []
        batch.extend([id] * num_atoms)
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edges.append((i, j))
            edges.append((j, i))
        
        edges = torch.tensor(edges, dtype = torch.long).T
        edges += atom_offset
        all_edge_index.append(edges)
        atom_offset += num_atoms
    
    all_edge_index = torch.cat(all_edge_index, dim = 1)
    batch = torch.tensor(batch, dtype=torch.long)
    return all_edge_index, batch

smiles = merge_csv['SMILES']
all_edge_index, batch = get_atom_edge_index(smiles)